## reading email

In [3]:
import imaplib 
import email 
from email.header import decode_header
import webbrowser
import os


In [15]:
from dotenv import load_dotenv 
load_dotenv()


True

In [31]:
app_key= os.getenv('app_password')

In [32]:
app_key

'rtjtoyqohzzxlfib'

In [21]:
import smtplib
from email.mime.text import MIMEText

# --- Configuration ---
subject = "Email Subject"
body = "This is the body of the text message"
sender = "ultopalto0@gmail.com"  # Your Gmail address
recipients = ["ultopalto0@gmail.com", "ujjwalvictus15@gmail.com"] # List of people to send to

# IMPORTANT: Replace the placeholder below with the 16-digit App Password you generated
# It will look something like: "abcd efgh ijkl mnop" (without spaces)
app_password = app_key

# --- Function to Send Email ---
def send_email(subject, body, sender, recipients, password):
    """
    Connects to Gmail's SMTP server and sends an email.
    """
    # Create the email message object
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)

    try:
        # Connect to the Gmail server using SSL
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
            # Log in using your email and the App Password
            smtp_server.login(sender, password)
            # Send the email
            smtp_server.sendmail(sender, recipients, msg.as_string())
        print("Message sent successfully!")
        
    except Exception as e:
        print(f"Failed to send email. Error: {e}")
        print("Please check the following:")
        print("1. Ensure your 'sender' email and 'app_password' are correct.")
        print("2. Check your internet connection.")

# --- Execute the function ---
send_email(subject, body, sender, recipients, app_password)


Message sent successfully!


In [23]:
import imaplib
import email
from email.header import decode_header



def fetch_recent_emails(username, password, server, num_emails=5):
    """
    Connects to Gmail's IMAP server, fetches recent emails, and parses them.
    """
    emails_list = []
    
    # Connect to the server using SSL for security
    try:
        mail = imaplib.IMAP4_SSL(server)
        # Login using your email and App Password
        mail.login(username, password)
        
        # Select  mailbox (e.g., 'inbox')
        mail.select('inbox')
        
        # Search all emails in the selected mailbox
        status, messages = mail.search(None, 'ALL')
        if status != 'OK':
            print("No messages found!")
            return []
            
        # Get  list of email IDs
        email_ids = messages[0].split()
        
        # Fetch the most recent emails
        latest_email_ids = email_ids[-num_emails:]
        
        print(f"Fetching the latest {len(latest_email_ids)} emails...")
        
        # Loop through the email IDs
        for email_id in reversed(latest_email_ids): # reversed to get newest first
            status, msg_data = mail.fetch(email_id, '(RFC822)')
            
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    # Parse the email content from bytes
                    msg = email.message_from_bytes(response_part[1])
                    
                    # Decode the subject and sender
                    subject, encoding = decode_header(msg['subject'])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else 'utf-8')
                        
                    from_ = msg.get('From')
                    
                    # Find the email body
                    body = ""
                    if msg.is_multipart():
                        # If the email has multiple parts, find the plain text part
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            if content_type == 'text/plain':
                                try:
                                    body = part.get_payload(decode=True).decode()
                                    break
                                except:
                                    pass
                    else:
                        # If it's not multipart, just get the payload
                        try:
                            body = msg.get_payload(decode=True).decode()
                        except:
                            pass
                            
                    emails_list.append({
                        'from': from_,
                        'subject': subject,
                        'body': body.strip()
                    })

    except Exception as e:
        print(f"Failed to fetch emails. Error: {e}")
    finally:
        # Always logout and close the connection
        if 'mail' in locals() and mail.state == 'SELECTED':
            mail.close()
            mail.logout()
            
    return emails_list


# --- Execute the function and print the results ---
if __name__ == "__main__":
    
        # --- Configuration ---
    # Your Gmail account credentials
    EMAIL_ACCOUNT = "ultopalto0@gmail.com"

    # Your 16-digit App Password (NO SPACES)
    APP_PASSWORD = app_key

    # The IMAP server for Gmail
    IMAP_SERVER = "imap.gmail.com"


    print("Connecting to your Gmail account to fetch emails...")
    my_emails = fetch_recent_emails(EMAIL_ACCOUNT, APP_PASSWORD, IMAP_SERVER)
    
    if my_emails:
        print("\n--- Here are your most recent emails ---\n")
        for i, mail_item in enumerate(my_emails):
            print(f"--- Email #{i+1} ---")
            print(f"From: {mail_item['from']}")
            print(f"Subject: {mail_item['subject']}")
            print("--- Body ---")
            print(mail_item['body'])
            print("-" * 20 + "\n")
    else:
        print("Could not retrieve any emails.")



Connecting to your Gmail account to fetch emails...
Fetching the latest 5 emails...

--- Here are your most recent emails ---

--- Email #1 ---
From: ultopalto0@gmail.com
Subject: Email Subject
--- Body ---
This is the body of the text message
--------------------

--- Email #2 ---
From: Google <no-reply@accounts.google.com>
Subject: Security alert
--- Body ---
[image: Google]
App password created to sign in to your account


ultopalto0@gmail.com
If you didn't generate this password for email_ai, someone else might be
using your account. Check and secure your account now.
Check activity
<https://accounts.google.com/AccountChooser?Email=ultopalto0@gmail.com&continue=https://myaccount.google.com/alert/nt/1753340716774?rfn%3D20%26rfnc%3D1%26eid%3D-4649410526140336145%26et%3D0>
You can also see security activity at
https://myaccount.google.com/notifications
You received this email to let you know about important changes to your
Google Account and services.
© 2025 Google LLC, 1600 Amphithea

In [27]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import os
from dotenv import load_dotenv

def fetch_emails_to_dataframe(username, password, server, num_emails=100):
    """
    Connects to an IMAP server, fetches recent emails, and returns them as a Pandas DataFrame.
    """
    emails_list = []
    mail = None  # Initialize mail to None

    try:
        # Connect to the server using SSL
        mail = imaplib.IMAP4_SSL(server)
        mail.login(username, password)
        mail.select('inbox')

        # Search for all emails
        status, messages = mail.search(None, 'ALL')
        if status != 'OK':
            print("No messages found!")
            return pd.DataFrame() # Return an empty DataFrame

        email_ids = messages[0].split()
        
        # Get the most recent email IDs up to the requested number
        latest_email_ids = email_ids[-num_emails:]
        
        print(f"Fetching the latest {len(latest_email_ids)} emails...")

        # Loop through the email IDs (reversed to get newest first)
        for email_id in reversed(latest_email_ids):
            status, msg_data = mail.fetch(email_id, '(RFC822)')
            
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    
                    # Decode subject, from, and date headers
                    subject, encoding = decode_header(msg['subject'])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else 'utf-8')
                        
                    from_ = msg.get('From')
                    date_ = msg.get('Date')
                    
                    # Extract the email body
                    body = ""
                    if msg.is_multipart():
                        for part in msg.walk():
                            if part.get_content_type() == 'text/plain' and part.get('Content-Disposition') is None:
                                try:
                                    body = part.get_payload(decode=True).decode()
                                    break
                                except:
                                    pass
                    else:
                        try:
                            body = msg.get_payload(decode=True).decode()
                        except:
                            pass
                            
                    emails_list.append({
                        'date': date_,
                        'from': from_,
                        'subject': subject,
                        'body': body.strip()
                    })

    except Exception as e:
        print(f"Failed to fetch emails. Error: {e}")
    finally:
        # Ensure logout and connection closure
        if mail and mail.state == 'SELECTED':
            mail.close()
            mail.logout()
            
    # Convert the list of dictionaries into a Pandas DataFrame
    return pd.DataFrame(emails_list)


# --- Main execution block ---
if __name__ == "__main__":
    # Load credentials from .env file
    load_dotenv()
    EMAIL_ACCOUNT = 'ultopalto0@gmail.com'
    IMAP_SERVER = "imap.gmail.com"

    if not EMAIL_ACCOUNT or not APP_PASSWORD:
        print("Error: GMAIL_EMAIL or GMAIL_APP_PASSWORD not found in .env file.")
    else:

        print(f"Connecting to {EMAIL_ACCOUNT} to fetch emails...")
        
        # Fetch emails and get the DataFrame
        emails_df = fetch_emails_to_dataframe(EMAIL_ACCOUNT, APP_PASSWORD, IMAP_SERVER, num_emails=100)
        
        if not emails_df.empty:
            print("\n--- Email data successfully loaded into a DataFrame ---")
            
            # Display DataFrame info
            print("\nDataFrame Info:")
            emails_df.info()
            
            # Display the first 5 emails from the DataFrame
            print("\nDataFrame Head (first 5 emails):")
            print(emails_df.head())

            # Optional: Save the DataFrame to a CSV file for later use
            try:
                emails_df.to_csv('recent_100_emails.csv', index=False)
                print("\nDataFrame saved to 'recent_100_emails.csv'")
            except Exception as e:
                print(f"\nCould not save DataFrame to CSV. Error: {e}")
        else:
            print("\nCould not retrieve any emails or an error occurred.")



Connecting to ultopalto0@gmail.com to fetch emails...
Fetching the latest 100 emails...

--- Email data successfully loaded into a DataFrame ---

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     100 non-null    object
 1   from     100 non-null    object
 2   subject  100 non-null    object
 3   body     100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB

DataFrame Head (first 5 emails):
                                    date  \
0  Thu, 24 Jul 2025 00:09:17 -0700 (PDT)   
1          Thu, 24 Jul 2025 07:05:16 GMT   
2          Thu, 24 Jul 2025 07:04:51 GMT   
3  Wed, 23 Jul 2025 23:31:23 -0700 (PDT)   
4  Wed, 23 Jul 2025 23:30:17 -0700 (PDT)   

                                                from  \
0                               ultopalto0@gmail.com   
1              Google <no-reply@accounts.google.com>   
2     

In [28]:
import pandas as pd 

In [30]:
pd.read_csv('recent_100_emails.csv')['subject']

0                                         Email Subject
1                                        Security alert
2                                        Security alert
3                                         Email Subject
4                Delivery Status Notification (Failure)
                            ...                        
95     Your 15-day free trial of Zoho Social ends today
96                 Celebrate 50 years of NAIDOC Week ☀️
97    Agentic AI Email Course Module 6 -Your Agentic...
98    What AI is doing to your job title (and how to...
99                                     PSYCHOLOGY FACTS
Name: subject, Length: 100, dtype: object